In [8]:
#import libraries
from keras.models import *
from keras.layers import *

#declare size of img
sz = 128

In [9]:
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))


# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2

classifier.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               3686528   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)              

In [10]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # categorical_crossentropy for more than 2

In [11]:
# Step 2 - Preparing the train/test data and training the model
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical')

Found 12845 images belonging to 27 classes.
Found 4268 images belonging to 27 classes.


In [12]:
#fitting the model
h = classifier.fit(training_set,validation_data=test_set,epochs = 10, batch_size=128,shuffle = True)

Epoch 1/10
1285/1285 [==============================] - 519s 400ms/step - loss: 1.9630 - accuracy: 0.3905 - val_loss: 0.4726 - val_accuracy: 0.8599
Epoch 2/10
1285/1285 [==============================] - 211s 164ms/step - loss: 0.9230 - accuracy: 0.6830 - val_loss: 0.1786 - val_accuracy: 0.9541
Epoch 3/10
1285/1285 [==============================] - 175s 136ms/step - loss: 0.6312 - accuracy: 0.7825 - val_loss: 0.0977 - val_accuracy: 0.9815
Epoch 4/10
1285/1285 [==============================] - 179s 139ms/step - loss: 0.4804 - accuracy: 0.8407 - val_loss: 0.0474 - val_accuracy: 0.9920
Epoch 5/10
1285/1285 [==============================] - 172s 134ms/step - loss: 0.3966 - accuracy: 0.8694 - val_loss: 0.0306 - val_accuracy: 0.9916
Epoch 6/10
1285/1285 [==============================] - 173s 135ms/step - loss: 0.3600 - accuracy: 0.8808 - val_loss: 0.0215 - val_accuracy: 0.9951
Epoch 7/10
1285/1285 [==============================] - 173s 135ms/step - loss: 0.2964 - accuracy: 0.9048 - val_

In [13]:
#Save the model
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model-bw.h5')
print('Weights saved')
